In [1]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import functools
from importlib import reload
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras
from keras.callbacks import Callback
from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Input
from keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.sparsity.keras import ConstantSparsity, PolynomialDecay, prune_low_magnitude

from src.harness.constants import Constants as C
from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.experiment import ExperimentData
from src.harness.model import create_lenet_300_100, create_masked_nn, create_pruned_lenet, save_model, load_model
from src.harness.pruning import create_pruning_callbacks, create_pruning_parameters
from src.harness.training import test_step, train, train_one_step, training_loop, TrainingRound
from src.harness.utils import count_params, set_seed

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Parameters

In [3]:
X_train, X_test, Y_train, Y_test = load_and_process_mnist()

num_epochs: int = 10
batch_size: int = 60
input_shape: tuple = X_train[0].shape
num_classes: int = 10

num_train_samples: int = X_train.shape[0]

end_step: int = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * num_epochs

pruning_parameters: dict = create_pruning_parameters(0.01, 0, end_step, 100)

## Building

In [4]:
# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
set_seed(0)

model: keras.Model = create_lenet_300_100(input_shape, num_classes)
model.summary()
model.trainable_variables

Model: "LeNet-300-100"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 300)            235500    
                                                                 
 dense_1 (Dense)             (None, 1, 100)            30100     
                                                                 
 dense_2 (Dense)             (None, 1, 10)             1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'dense/kernel:0' shape=(784, 300) dtype=float32, numpy=
 array([[ 0.00233377, -0.02841079,  0.05042759, ...,  0.04471111,
          0.0469005 , -0.00633286],
        [ 0.04599051,  0.07108735, -0.06869254, ...,  0.0193451 ,
         -0.03234061,  0.03169315],
        [ 0.04332277,  0.00188898, -0.05098354, ...,  0.05931461,
          0.0038614 , -0.06489589],
        ...,
        [ 0.06133612, -0.00425664, -0.03556142, ...,  0.05923072,
         -0.04446548,  0.03342723],
        [ 0.02262612, -0.01283204,  0.03786094, ..., -0.03319833,
          0.0259892 , -0.02313284],
        [ 0.00880335,  0.00193495, -0.07007851, ..., -0.05422467,
         -0.0134997 , -0.03317983]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(300,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [5]:
mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)
mask_model.summary()
mask_model.trainable_variables

Model: "Pruned_LeNet-300-100"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense_3  (None, 1, 300)           470702    
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_4  (None, 1, 100)           60102     
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_5  (None, 1, 10)            2012      
  (PruneLowMagnitude)                                            
                                                                 
Total params: 532,816
Trainable params: 266,610
Non-trainable params: 266,206
_________________________________________________________________


[<tf.Variable 'dense_3/kernel:0' shape=(784, 300) dtype=float32, numpy=
 array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_3/bias:0' shape=(300,) dtype=float32, numpy=
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

## Pruning

## Training

In [6]:
# Test single step of training

# Define the optimizer outside of the function
optimizer = C.OPTIMIZER()

# Create model and masked model
model: keras.Model = create_lenet_300_100(input_shape, num_classes)
mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)

loss, accuracy = train_one_step(model, mask_model, X_train, Y_train, optimizer)
print(f'Loss: {loss:.6f}, Accuracy: {accuracy:.6g}')

Loss: 2.397225, Accuracy: 0.0663


In [7]:
# Try getting test accuracies with `test_step`
test_loss: tf.keras.metrics.Metric = C.LOSS_FUNCTION()
test_accuracy: tf.keras.metrics.Metric = tf.keras.metrics.CategoricalAccuracy()

loss, accuracy = test_step(model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Test Loss: 95.3614501953125
Test Accuracy: 0.16609999537467957


In [8]:
# Testing `training_loop` function

# Define the optimizer outside of the function
optimizer = C.OPTIMIZER()

epochs: int = 2

# Create model and masked model
model: keras.Model = create_lenet_300_100(input_shape, num_classes)
mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)

trained_model, training_round = training_loop(0, model, mask_model, load_and_process_mnist, epochs, batch_size, optimizer=optimizer)
print(f'Took {np.sum(training_round.test_accuracies != 0)} / {epochs} epochs')
print(f'Ended with a best training accuracy of {np.max(training_round.train_accuracies) * 100:.2f}% and test accuracy of {np.max(training_round.test_accuracies) * 100:.2f}%')

Took 2 / 2 epochs
Ended with a best training accuracy of 35.66% and test accuracy of training accuracy of 54.87%


In [9]:
# Testing `train` function

# Create model and masked model
model: keras.Model = create_lenet_300_100(input_shape, num_classes)
mask_model: keras.Model = create_masked_nn(create_pruned_lenet, input_shape, num_classes, pruning_parameters)

trained_model, pruned_mask_model, training_round = train(0, 0, model, mask_model, load_and_process_mnist)

test_accuracy: tf.keras.metrics.Metric = tf.keras.metrics.CategoricalAccuracy()

print(f'Took {np.sum(training_round.test_accuracies != 0)} / {C.TRAINING_EPOCHS} epochs')
print(f'Ended with a best training accuracy of {np.max(training_round.train_accuracies) * 100:.2f}% and test accuracy of training accuracy of {np.max(training_round.test_accuracies) * 100:.2f}%')

loss, accuracy = test_step(trained_model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Early stopping initiated
Directory 'models/model_0/trial0/weights' already exists.
Directory 'models/model_0/trial0/masks' already exists.
Took 11 / 30 epochs
Ended with a best training accuracy of 55.14% and test accuracy of training accuracy of 63.33%
Test Loss: 1.2082486152648926
Test Accuracy: 0.4763000011444092


In [10]:
# Test loading the model back
loaded_model: keras.Model = load_model(0, 0)

loss, accuracy = test_step(loaded_model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Test Loss: 1.2082486152648926
Test Accuracy: 0.4763000011444092


In [11]:
print(f'Nonzero parameters after training:')
print(count_params(model)[1])

Nonzero parameters after training:
266610
